In [14]:
import numpy as np
import pandas as pd 
import re
from keras.preprocessing.text import Tokenizer 
import nltk
from nltk import download
download('stopwords')
from nltk.corpus import stopwords


import json
import gzip
import os
import sys
import argparse
#import panda as pd
import matplotlib.pyplot as plt
import re
# Library for boxplots
import seaborn as sns


from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings

#from tensorflow.keras.layers import Attention
#from attention import AttentionLayer

#from keras_self_attention import SeqSelfAttention
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Malo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [65]:
CURRENCIES = {
    "$": "USD", "zł": "PLN", "£": "GBP", "¥": "JPY", "฿": "THB", "₡": "CRC", "₦": "NGN","₩": "KRW",
    "₪": "ILS", "₫": "VND", "€": "EUR", "₱": "PHP", "₲": "PYG", "₴": "UAH", "₹": "INR",}
CURRENCY_REGEX = re.compile(
    "({})+".format("|".join(re.escape(c) for c in CURRENCIES.keys())))

EMAIL_REGEX = re.compile(
    r"(?:^|(?<=[^\w@.)]))([\w+-](\.(?!\.))?)*?[\w+-]@(?:\w-?)*?\w+(\.([a-z]{2,})){1,3}(?:$|(?=\b))",
    flags=re.IGNORECASE | re.UNICODE,)

# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions =          {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have", "i've": "i have"}
def clean_text(text, remove_stopwords = True):
    
    text = text.lower()
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                print("contract")
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
        
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = EMAIL_REGEX.sub(' ',text)
    text = CURRENCY_REGEX.sub(' ',text)
    text = ' '.join([contractions[t] if t in contractions else t for t in text.split(" ")])    
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r"'s\b","", text)
    text = re.sub(r'&amp;', '', text) 
    
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text



In [66]:
# A remplacer par "train"  "val"   "test"
kind_data="train"

file_names = [file for file in os.listdir(os.path.join("data",kind_data,"g")) if ".txt" not in file]
for file_name in file_names:
    listJSON = readData(os.path.join("data",kind_data,"g"),file_name)
    #JSONlist=filterChars(os.path.join("data",kind_data,"g"),file_name,listJSON)
    for i in range(len(listJSON)) :
        abstract = listJSON[i]['abstract']
        description = listJSON[i]['description']
        #print(len(abstract))
        #print(abstract)
        txt2=clean_text(abstract, remove_stopwords = True)
        
        
        break
    break
        

1241
methods and systems are provided for obtaining information related to a customer service location and directions for routing a service technician from one customer service location to another . one embodiment includes requesting at least one set of coordinates associated with the customer service location ; accessing a technician server to direct a global satellite positioning system to obtain the set of coordinates for the customer service location ; obtaining the coordinates and updating one or more databases with said coordinates . the coordinates may include at least one of a latitude and a longitude associated with the customer service location . another embodiment includes obtaining through a technician server at least one set of “ from ” coordinates associated with the first customer service location and at least one set of “ to ” coordinates associated with the second customer location ; transmitting the “ from ” and “ to ” coordinates to a mapping system ; and , generatin

In [67]:
print(len(txt2))
print(txt2)

957
methods systems provided obtaining information related customer service location directions routing service technician one customer service location another one embodiment includes requesting least one set coordinates associated customer service location accessing technician server direct global satellite positioning system obtain set coordinates customer service location obtaining coordinates updating one databases said coordinates coordinates may include least one latitude longitude associated customer service location another embodiment includes obtaining technician server least one set “ ” coordinates associated first customer service location least one set “ ” coordinates associated second customer location transmitting “ ” “ ” coordinates mapping system generating directions mapping system based “ ” “ ” coordinates least one sets coordinates includes latitude longitude data system computer readable media embodiments methods also provided


In [46]:


def readData(path,filename):
    json_obj_list=[]
    with open(os.path.join(path,filename),'r') as fin:
        for row in fin:
            json_obj_list.append(json.loads(row))
    return json_obj_list


def filterChars(path,filename,JSONlist):
    cachedstopwords = stopwords.words("english")
    if os.path.exists(path+'new_'+filename+'.txt')==True:
        os.remove(path+'new_'+filename+'.txt')
    new_f= open(os.path.join(path,'new_'+filename)+'.txt','a')
    for i in range(len(JSONlist)) :
        abstract = JSONlist[i]['abstract']
        description = JSONlist[i]['description']
            
        abstract1 = abstract.lower()
        description1 = description.lower()
            
        abstract2 = re.sub('[^a-zA-Z ]',' ',abstract1)
        description2 = re.sub('[^a-zA-Z ]',' ',description1)
            
        abstract3 = re.sub(r'(?:^| )\w(?:$| )', ' ', abstract2).strip()
        description3 = re.sub(r'(?:^| )\w(?:$| )', ' ', description2).strip()
            
        abstract4 = ' '.join([word for word in abstract3.split() if word not in cachedstopwords])
        description4 = ' '.join([word for word in description3.split() if word not in cachedstopwords])
            
        abstract5=re.sub(' +',' ',abstract4)
        description5=re.sub(' +',' ',description4)
            
        JSONlist[i]['abstract']=abstract5
        JSONlist[i]['description']=description5
        updatedJSON={"publication_number":JSONlist[i]['publication_number'],"abstract":abstract5,"description":description5}
        new_f.write(json.dumps(updatedJSON))
    print("Wrote file "+new_f.name)
    new_f.close()
    return JSONlist


In [ ]:
# A remplacer par "train"  "val"   "test"
kind_data="train"

file_names = [file for file in os.listdir(os.path.join("data",kind_data,"g")) if ".txt" in file]
for file_name in file_names:
    listJSON = readData(os.path.join("data",kind_data,"g"),file_name)
    #JSONlist=filterChars(os.path.join("data",kind_data,"g"),file_name,listJSON)